In [1]:
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext, ClientCredential
from office365.sharepoint.files.file import File
import json
import os
from dotenv import load_dotenv

In [2]:
def get_hubpage_content(site_url, username, password, page_url):
	auth_ctx = AuthenticationContext(site_url)
	if auth_ctx.acquire_token_for_user(username, password):
		credentials = ClientCredential(username, password)
		ctx = ClientContext(site_url, auth_ctx)
		response = (
			File.from_url(page_url)
			.with_credentials(credentials)
			.download(ctx)
			.execute_query()
		)
		page_content = response.content
		try:
			content = json.loads(page_content)
			return content
		except Exception:
			return page_content.decode("utf-8")
	else:
		raise Exception("Authentication failed")

In [3]:
load_dotenv()
SITE_URL = "https://cehacuk.sharepoint.com/sites/hub-pollution"
USERNAME = os.environ.get("USER")
PASSWORD = os.environ.get("PASSWORD")
PAGE_URL = "https://cehacuk.sharepoint.com/sites/hub-pollution/SitePages/Research-Software-Engineers-(RSEs).aspx"

content = get_hubpage_content(SITE_URL, USERNAME, PASSWORD, PAGE_URL)
print(content)

ValueError: {"error":"invalid_client","error_description":"AADSTS70002: Client application name 'matcoo@ceh.ac.uk@7f025136-39f3-4f2c-a3c6-cbb1ce21b36a' is not valid or the credentials used to authenticate the client could not be understood by the server. Trace ID: a6716399-5df7-4bd4-a7c5-3418faee2100 Correlation ID: effbb024-adca-41ad-b564-050faff0d2fc Timestamp: 2025-01-27 10:59:03Z","error_codes":[70002],"timestamp":"2025-01-27 10:59:03Z","trace_id":"a6716399-5df7-4bd4-a7c5-3418faee2100","correlation_id":"effbb024-adca-41ad-b564-050faff0d2fc","error_uri":"https://accounts.accesscontrol.windows.net/error?code=70002"}

In [4]:
from requests_ntlm import HttpNtlmAuth
import requests


def access_sharepoint(site_url, username, password):
	# Create NTLM auth object
	auth = HttpNtlmAuth(username, password)

	# Make request with NTLM auth
	response = requests.get(site_url, auth=auth, verify=True)

	return response


# Example usage
site_url = "https://cehacuk.sharepoint.com/sites/hub-pollution/SitePages/Research-Software-Engineers-(RSEs).aspx"
username = os.environ.get("USER")  # or username@domain.com
password = os.environ.get("PASSWORD")

try:
	response = access_sharepoint(site_url, username, password)
	print(f"Status: {response.status_code}")
	print(f"Content: {response.text[:200]}")  # First 200 chars
except requests.exceptions.RequestException as e:
	print(f"Error: {e}")

Status: 403
Content: 403 FORBIDDEN
